## Docs
[Julia Bifurcation Analysis BifurcationKit](https://docs.juliahub.com/BifurcationKit/I1INQ/0.1.0/tutorials1/#Temperature-model-1)

In [ ]:
using Pkg
#Pkg.add("DifferentialEquations")
using Conda
using WebIO
using Interact
using DifferentialEquations
using DiffEqFlux
using DiffEqSensitivity, OrdinaryDiffEq, Zygote
using Plots
using Flux, Optim, Test



In [ ]:
#Pkg.add("Optimization")
#Pkg.add("OptimizationPolyalgorithms")
using Optimization, OptimizationPolyalgorithms

## 1. General Approximation using Neural Network

## True Assumption:
$$
\ddot{x}+b\dot{x}+w_{n}^{2}x+\mu x^3+vx^5+\rho x^7=\delta _{st}w_{n}^{2}\cos \left( wt \right) 
$$

$$
\left\{ \begin{array}{l}
	\dot{x}=y\\
	\dot{y}=-by-w_{n}^{2}x-\mu x^3-vx^5-\rho x^7+\delta _{st}w_{n}^{2}\cos \left( wt \right)\\
\end{array} \right. 
$$


## Approximate
$$
\ddot{x}+b\dot{x}+w_{n}^{2}x+\mu x^3=\delta _{st}w_{n}^{2}\cos \left( wt \right)+NN\left( x,t \right)
$$


$$
\left\{ \begin{array}{l}
	\dot{x}=y\\
	\dot{y}=-by-w_{n}^{2}x-\mu x^3+\delta _{st}w_{n}^{2}\cos \left( wt \right)+NN\left( x,t \right)\\
\end{array} \right. 
$$

### 1.1 Genrate Simulated data according to True Assumption

In [ ]:
dx_init = 0.0
x_init = 1.0

datasize = 30
tspan = (0.0,10)
#parameter value from the Robust paper
param = [0.3159,1,1.499,-0.3921,0.0422,1,round(2*pi*19.95,digits = 3)]

function Duffing(u,p,t)
    b,wn_2,mu,v,rho,delta_st,w = param
    x,y = u
    du1 = y
    du2 = -b*y - wn_2*x - mu*x^3 - v*x^5 - rho*x^7 + delta_st * wn_2 * cos(w*t)
    du = [du1,du2]
end



t = range(tspan[1],tspan[2],length=datasize)

prob = ODEProblem(Duffing,[x_init,dx_init],tspan)
sol = Array(solve(prob,Tsit5(),saveat=t))


plot(t,sol[1,:],label = "x")
scatter!(t,sol[1,:],label = "x_data")
plot!(t,sol[2,:],label = "dx")
scatter!(t,sol[2,:],label = "dx_data",title = "7th Order True Data")






In [ ]:
sol

In [ ]:
#from the Robust paper

function Duffing_Approx(u,p,t)
    b,wn_2,mu,v,rho,delta_st,w = param
    x,y = u
    du1 = y
    #there is no v and rho
    du2 = -b*y - wn_2*x - mu*x^3 + delta_st * wn_2 * cos(w*t)
    du = [du1,du2]
end



prob_approx = ODEProblem(Duffing_Approx,[x_init,dx_init],tspan)
sol_approx = Array(solve(prob_approx,Tsit5(),saveat=t))


plot(t,sol_approx[1,:],label = "x")
scatter!(t,sol_approx[1,:],label = "x_data")
plot!(t,sol_approx[2,:],label = "dx")
scatter!(t,sol_approx[2,:],label = "dx_data",title = "3th Order Data using for Approx")


In [ ]:
plot(t,sol_approx[1,:],label = "3th x")
scatter!(t,sol_approx[1,:],label = "3th x_data")
plot!(t,sol[1,:],label = "7th x")
scatter!(t,sol[1,:],label = "7th x_data",title = "3th and 7th Order Data")

### 1.2 Fit a simple Neural ODE to the 7th Order data above

In [ ]:
dudt = Flux.Chain(
             Dense(2,50,tanh),
             Dense(50,2))
n_ode = NeuralODE(dudt,tspan,Tsit5(),saveat=t,reltol=1e-7,abstol=1e-9)
ps = Flux.params(n_ode)

In [ ]:
u0 = [x_init,dx_init]
pred = n_ode(u0) # Get the prediction using the correct initial condition
scatter(t,sol[1,:],label="x_data",color = "red")
scatter!(t,pred[1,:],label="x_prediction",color = "blue")

In [ ]:
function predict_n_ode()
  n_ode(u0)
end
loss_n_ode() = sum(abs2,reshape(sol .- predict_n_ode(),:))

In [ ]:
data = Iterators.repeated((), 100)
opt = ADAM(0.1)
cb = function () #callback function to observe training
  display(loss_n_ode())
  # plot current prediction against data
  cur_pred = predict_n_ode()
  pl = scatter(t,sol[1,:],label="x_data",color = "red")
  scatter!(pl,t,cur_pred[1,:],label="x_prediction",color = "blue")
  display(plot(pl))
end

# Display the ODE with the initial parameter values.
cb()

Flux.train!(loss_n_ode, ps, data, opt, cb = cb)



### The prediction above seems to be stuck in the local optimal, so we should connect with existing knowledge!
### 1.3 Using the known term of Approximation
### 1.3.1 Using NN to Approximate the residual of these two data

In [ ]:
dudt2 = Flux.Chain(
             Dense(2,50,tanh),
             Dense(50,2))
n_ode2 = NeuralODE(dudt2,tspan,Tsit5(),saveat=t)
ps2 = Flux.params(n_ode2)

In [ ]:
pred2 = n_ode2([0.01,0.01]) # Get the prediction using the correct initial condition
scatter(t,sol_approx[1,:]-sol[1,:],label="Residual",color = "red")
scatter!(t,pred2[1,:],label="prediction",color = "blue")

In [ ]:
function predict_n_ode2()
  Array(n_ode2([0.01,0.01]))
end
loss_n_ode2() = sum(abs2,reshape((sol_approx-sol) .- predict_n_ode2(),:))

In [ ]:
data2 = Iterators.repeated((), 200)
opt = ADAM(0.05)
cb2 = function () #callback function to observe training
  display(loss_n_ode2())
  # plot current prediction against data
  cur_pred2 = predict_n_ode2()
  pl = scatter(t,sol_approx[1,:]-sol[1,:],label="Residual",color = "red")
  scatter!(pl,t,cur_pred2[1,:],label="prediction",color = "blue")
  display(plot(pl))
end

# Display the ODE with the initial parameter values.
cb2()

Flux.train!(loss_n_ode2, ps2, data2, opt, cb = cb2)

### Still local minima, so we may try some other methods
### 1.3.2 using segment to fit in order to go out of local minimal (7th-order)

In [ ]:
using Lux
using Random
rng = Random.default_rng()
using DiffEqFlux: group_ranges

In [ ]:
nn = Lux.Chain(
                  Lux.Dense(2, 30, tanh),
                  Lux.Dense(30, 2))
p_init, st = Lux.setup(rng, nn)
tsteps = t
neuralode = NeuralODE(nn, tspan, Tsit5(), saveat = tsteps)
prob_node = ODEProblem((u,p,t)->nn(u,p,st)[1], u0, tspan, Lux.ComponentArray(p_init))

function plot_multiple_shoot(plt, preds, group_size)
    step = group_size-1
    ranges = group_ranges(datasize, group_size)

    for (i, rg) in enumerate(ranges)
        plot!(plt, tsteps[rg], preds[i][1,:], markershape=:circle, label="Group $(i)")
    end
end

In [ ]:
anim = Plots.Animation()
iter = 0
callback = function (p, l, preds; doplot = true)
  display(l)
  global iter
  iter += 1
  if doplot && iter%1 == 0
    # plot the original data
    plt = scatter(tsteps, sol[1,:], label = "Data")

    # plot the different predictions for individual shoot
    plot_multiple_shoot(plt, preds, group_size)

    frame(anim)
    display(plot(plt))
  end
  return false
end

In [ ]:
group_size = 3
continuity_term = 200

function loss_function(data, pred)
    return sum(abs2, data - pred)
end

function loss_multiple_shooting(p)
    return multiple_shoot(p, sol, tsteps, prob_node, loss_function, Tsit5(),
                          group_size; continuity_term)
end

adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x,p) -> loss_multiple_shooting(x), adtype)
optprob = Optimization.OptimizationProblem(optf, Lux.ComponentArray(p_init))
res_ms = Optimization.solve(optprob, PolyOpt(),
                                callback = callback,maxiters = 100)

### 1.3.3 using segment to fit the residual (7th-3th) in order to go out of local minimal 

In [ ]:
nn = Lux.Chain(
                  Lux.Dense(2, 50, tanh),
                  Lux.Dense(50, 2))
p_init, st = Lux.setup(rng, nn)
tsteps = t
neuralode = NeuralODE(nn, tspan, Tsit5(), saveat = tsteps)
prob_node = ODEProblem((u,p,t)->nn(u,p,st)[1], u0, tspan, Lux.ComponentArray(p_init))

function plot_multiple_shoot(plt, preds, group_size)
    step = group_size-1
    ranges = group_ranges(datasize, group_size)

    for (i, rg) in enumerate(ranges)
        plot!(plt, tsteps[rg], preds[i][1,:], markershape=:circle, label="Group $(i)")
    end
end

In [ ]:
anim = Plots.Animation()
iter = 0
callback = function (p, l, preds; doplot = true)
  display(l)
  global iter
  iter += 1
  if doplot && iter%1 == 0
    # plot the original data
    plt = scatter(tsteps, sol_approx[1,:]-sol[1,:], label = "Data")

    # plot the different predictions for individual shoot
    plot_multiple_shoot(plt, preds, group_size)

    frame(anim)
    display(plot(plt))
  end
  return false
end

In [ ]:
group_size = 3
continuity_term = 200

function loss_function(data, pred)
    return sum(abs2, data - pred)
end

function loss_multiple_shooting(p)
    return multiple_shoot(p, sol_approx-sol, tsteps, prob_node, loss_function, Tsit5(),
                          group_size; continuity_term)
end

adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x,p) -> loss_multiple_shooting(x), adtype)
optprob = Optimization.OptimizationProblem(optf, Lux.ComponentArray(p_init))
res_ms = Optimization.solve(optprob, PolyOpt(),
                                callback = callback,maxiters = 100)

## 2. Introduce Electromagnetics

## True Assumption:
$$
\left\{ \begin{array}{l}
	\ddot{x}+b\dot{x}+w_{n}^{2}x+\mu x^3+vx^5+\rho x^7+\theta z=\delta _{st}w_{n}^{2}\cos \left( wt \right)\\
	\dot{z}=\theta \dot{x}-Rz\\
\end{array} \right. 
$$

$$
\left\{ \begin{array}{l}
	\dot{x}=y\\
	\dot{y}=-by-w_{n}^{2}x-\mu x^3-vx^5-\rho x^7-\theta z+\delta _{st}w_{n}^{2}\cos \left( wt \right)\\
	\dot{z}=\theta y-Rz\\
\end{array} \right. 
$$



## Approximate
$$
\left\{ \begin{array}{l}
	\ddot{x}+b\dot{x}+w_{n}^{2}x+\mu x^3+\theta z=\delta _{st}w_{n}^{2}\cos \left( wt \right) +NN\left( x,t \right)\\
	\dot{z}=\theta \dot{x}-Rz\\
\end{array} \right. 
$$
$$
\left\{ \begin{array}{l}
	\dot{x}=y\\
	\dot{y}=-by-w_{n}^{2}x-\mu x^3-\theta z+\delta _{st}w_{n}^{2}\cos \left( wt \right) +NN\left( x,t \right)\\
	\dot{z}=\theta y-Rz\\
\end{array} \right. 
$$


In [ ]:
dx_init = 0.0
x_init = 1.0
z_init = 0.0

datasize = 30
tspan = (0.0,10)
#parameter value from the Robust paper
param = [0.3159,1,1.499,-0.3921,0.0422,1,round(2*pi*19.95,digits = 3),4.5,12.5]

function Duffing(u,p,t)
    b,wn_2,mu,v,rho,delta_st,w,theta,R = param
    x,y,z = u
    du1 = y
    du2 = -b*y - wn_2*x - mu*x^3 - v*x^5 - rho*x^7 -theta*z + delta_st * wn_2 * cos(w*t)
    du3 = theta*10e4*y-R*10e4*z
    du = [du1,du2,du3]
end



t = range(tspan[1],tspan[2],length=datasize)

prob = ODEProblem(Duffing,[x_init,dx_init,z_init],tspan)
sol = Array(solve(prob,TRBDF2(),saveat=t))


plot(t,sol[1,:],label = "x")
scatter!(t,sol[1,:],label = "x_data")
plot!(t,sol[2,:],label = "y")
scatter!(t,sol[2,:],label = "y_data")
plot!(t,sol[3,:],label = "z")
scatter!(t,sol[3,:],label = "z_data",title = "7th Order True Data")

In [ ]:
function Duffing_Approx(u,p,t)
    b,wn_2,mu,v,rho,delta_st,w,theta,R = param
    x,y,z = u
    du1 = y
    du2 = -b*y - wn_2*x - mu*x^3  -theta*z/R^2 + delta_st * wn_2 * cos(w*t)
    du3 = theta*10e2*y-R*10e2*z
    du = [du1,du2,du3]
end



prob_approx = ODEProblem(Duffing_Approx,[x_init,dx_init,z_init],tspan)
sol_approx = Array(solve(prob_approx,TRBDF2(),saveat=t))


plot(t,sol_approx[1,:],label = "x")
scatter!(t,sol_approx[1,:],label = "x_data")
plot!(t,sol_approx[2,:],label = "y")
scatter!(t,sol_approx[2,:],label = "y_data")
plot!(t,sol_approx[3,:],label = "z")
scatter!(t,sol_approx[3,:],label = "z_data",title = "3th Order Data using for Approx")

In [ ]:
plot(t,sol_approx[1,:]-sol[1,:],label = "x")
scatter!(t,sol_approx[1,:]-sol[1,:],label = "x_data")
plot!(t,sol_approx[2,:]-sol[2,:],label = "y")
scatter!(t,sol_approx[2,:]-sol[2,:],label = "y_data")
plot!(t,sol_approx[3,:]-sol[3,:],label = "z")
scatter!(t,sol_approx[3,:]-sol[3,:],label = "z_data",title = "Residual")

### 2.1 using segment to fit the residual (7th-3th)

In [ ]:
nn = Lux.Chain(
                  Lux.Dense(3, 50, relu),
                  Lux.Dense(50, 3))
p_init, st = Lux.setup(rng, nn)
tsteps = t
neuralode = NeuralODE(nn, tspan, TRBDF2(), saveat = tsteps)
prob_node = ODEProblem((u,p,t)->nn(u,p,st)[1], [1.0,0.0,0.0], tspan, Lux.ComponentArray(p_init))

function plot_multiple_shoot(plt, preds, group_size)
    step = group_size-1
    ranges = group_ranges(datasize, group_size)

    for (i, rg) in enumerate(ranges)
        plot!(plt, tsteps[rg], preds[i][1,:], markershape=:circle, label="Group $(i)")
    end
end

In [ ]:
anim = Plots.Animation()
iter = 0
callback = function (p, l, preds; doplot = true)
  display(l)
  global iter
  iter += 1
  if doplot && iter%1 == 0
    # plot the original data
    plt = scatter(tsteps, sol_approx[1,:]-sol[1,:], label = "Data")

    # plot the different predictions for individual shoot
    plot_multiple_shoot(plt, preds, group_size)

    frame(anim)
    display(plot(plt))
  end
  return false
end

In [ ]:
group_size = 3
continuity_term = 200

function loss_function(data, pred)
    return sum(abs2, data - pred)
end

function loss_multiple_shooting(p)
    return multiple_shoot(p, sol_approx-sol, tsteps, prob_node, loss_function, Tsit5(),
                          group_size; continuity_term)
end

adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x,p) -> loss_multiple_shooting(x), adtype)
optprob = Optimization.OptimizationProblem(optf, Lux.ComponentArray(p_init))
res_ms = Optimization.solve(optprob, PolyOpt(),
                                callback = callback,maxiters = 100)